# Schematics
Below, I create some schematics to help understand the theory for how temperature distribution changes with warming.

In [1]:
import sys
import os
# REMOTE - So can access functions in isca_tools which is in home/Isca directory
# sys.path.append(os.path.join(os.environ['HOME'], 'Isca'))
# LOCAL - So can access functions in isca_tools which is in StAndrews/Isca
sys.path.append(os.environ['PWD'])
import isca_tools
from isca_tools.utils.moist_physics import clausius_clapeyron_factor, sphum_sat, moist_static_energy
from isca_tools.convection.base import convection_neutral_profile, lcl_temp_bolton, dry_profile_pressure, lapse_moist, lapse_dry, rh_from_sphum
from isca_tools.utils.constants import L_v, c_p, g, R
from isca_tools.utils.stats import z_score_from_confidence_interval
from isca_tools.utils import area_weight_mean_lat, area_weighting
from isca_tools.thesis.adiabat_theory import get_theory_prefactor_terms, get_temp_adiabat, get_gamma_factors
from isca_tools.papers.byrne_2021 import get_quant_ind
from isca_tools.plot import label_subplots
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import xarray as xr
import scipy.optimize
from tqdm import tqdm
import os
import numpy_indexed
plt.style.use('/Users/joshduffield/Documents/StAndrews/Isca/jobs/tau_sweep/aquaplanet/publish_figures/publish.mplstyle')

In [2]:
# If true, will save all figures to desktop - option to save specific figures later on.
save_fig = False
# if publishing figure, use high dpi
publish_fig = True

ar = 4/3        # aspect ratio (width/height)
# Details required for Journal of Climate Figures
low_dpi = 100
dpi = {'monochrome': 1100, 'combination': 800, 'halftone': 300}
width = {'one_col': 3.2, 'two_col': 5.5}        # width in inches 
save_pad_inches = 0.05

# Default parameters
default_colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
ax_linewidth = plt.rcParams['axes.linewidth']

In [3]:
# Get surface and free tropospheric pressure values in Pa.
pressure_ft_approx = 500        # approximate pressure of free troposphere in hPa
exp_dir = 'tau_sweep/aquaplanet/depth=1/k=1/'
pressure_surface, pressure_ft = isca_tools.load_dataset(exp_dir).pfull.sel(pfull=[np.inf, pressure_ft_approx], method='nearest').to_numpy() * 100

## Theory Terms
Below I plot a schematic for each term in the new formula - 4 in total.
For each, I isolate the effect of one contribution.

In [38]:
pressure_schematic = np.linspace(400*100, pressure_surface, 100)

def get_temp_surf_from_temp_adiabat(temp_adiabat, r_surface = None, pressure_surface=pressure_surface, pressure_ft=pressure_ft, 
                                    guess_temp_surf=280):
    def fit_func(temp_surf):
        temp_lcl = lcl_temp_bolton(temp_surf, r_surface*100)
        return convection_neutral_profile(float(temp_surf), pressure_surface, float(temp_lcl), np.array([pressure_ft])) - temp_adiabat
    return float(scipy.optimize.fsolve(fit_func, guess_temp_surf))

def schematic_plot(temp_cold, rh_cold, temp_adiabat_change_cold, r_change_cold = 0, 
                   temp_hot = None, rh_hot = None, temp_adiabat_change_hot = None, r_change_hot=0, 
                   pressure_surface = pressure_surface, pressure_ft=pressure_ft, 
                   pressure_levels=pressure_schematic, include_diff_plot = False, sphum_cold=None, sphum_hot = None):
    """
    Method to plot schematic showing how the temperature profile of a cold (blue) and hot (red) day changes under warming, with given 
    constraints. Idea is to isolate each term in equation for change in surface temperature.
    
    Args:
        temp_cold: float
            Surface temperature of cold day in reference climate [K].
        rh_cold: float
            Surface relative humidity of cold day in reference climate. If `None`, `sphum_cold` is used.
        temp_adiabat_change_cold: float
            Change of adiabatic temperature of cold day at `pressure_ft` with warming [K].
        r_change_cold: float
            Change of surface relative humidity of cold day with warming.
        temp_hot: float
            Surface temperature of hot day in reference climate [K]. If not given, set to `temp_cold`.
        rh_hot: float
            Surface relative humidity of hot day in reference climate. If not given, set to `rh_cold`.
        temp_adiabat_change_hot: float
            Change of adiabatic temperature of hot day at `pressure_ft` with warming [K]. If not given, set to `temp_adiabat_change_cold`.
        r_change_hot: float
            Change of surface relative humidity of hot day with warming.
        pressure_surface: float
            Near-surface pressure [hPa].
        pressure_ft: float
            Free tropospheric pressure [hPa]
        pressure_levels: float [n_levels]
            Pressure levels to plot temperature profile at.
        include_diff_plot: bool
            If `True`, the figure will have a second axis, plotting the temperature profile anomaly for each climate. 
            I.e. red - blue line for each climate.
        sphum_cold: float
            Surface specific humidity of cold day in reference climate [kg/kg].
        sphum_hot: float
            Surface specific humidity of hot day in reference climate. If not given, set to `sphum_cold`.
    """
    n_climates = 2      # function assumes there are two climates and a cold and hot day (as indicated by dict) in both.
    temp_surf = {'cold': np.zeros(n_climates), 'hot': np.zeros(n_climates)}
    rh = {'cold': np.zeros(n_climates), 'hot': np.zeros(n_climates)}
    temp_lcl = {'cold': np.zeros(n_climates), 'hot': np.zeros(n_climates)}
    pressure_lcl = {'cold': np.zeros(n_climates), 'hot': np.zeros(n_climates)}
    temp_adiabat = {'cold': np.zeros(n_climates), 'hot': np.zeros(n_climates)}
    temp_profile = {'cold': np.zeros((n_climates, len(pressure_levels))), 'hot': np.zeros((n_climates, len(pressure_levels)))}
    
    temp_surf['cold'][0] = temp_cold
    temp_surf['hot'][0] = temp_cold if temp_hot is None else temp_hot
    if rh_cold is None:
        # Use specific humidity if rh is None
        rh_cold = sphum_cold / sphum_sat(temp_cold, pressure_surface) 
        rh_hot = (sphum_cold if sphum_hot is None else sphum_hot) / sphum_sat(temp_surf['hot'][0], pressure_surface) 
    rh['cold'][0] = rh_cold
    rh['hot'][0] = rh_cold if rh_hot is None else rh_hot
        
        
    for key in temp_surf:
        temp_lcl[key][0] = np.clip(lcl_temp_bolton(temp_surf[key][0], rh[key][0] * 100), 0, temp_surf[key][0])
        temp_adiabat[key][0] = float(convection_neutral_profile(temp_surf[key][0], pressure_surface, temp_lcl[key][0], np.array([pressure_ft])))
    
    # Shift to warmer climate
    if temp_adiabat_change_hot is None:
        temp_adiabat_change_hot = temp_adiabat_change_cold
    temp_adiabat_change = {'cold': temp_adiabat_change_cold, 'hot': temp_adiabat_change_hot}
    rh_change = {'cold': r_change_cold, 'hot': r_change_hot}
    for key in temp_surf:
        rh[key][1] = rh[key][0] + rh_change[key]
        temp_adiabat[key][1] = temp_adiabat[key][0] + temp_adiabat_change[key]
        temp_surf[key][1] = get_temp_surf_from_temp_adiabat(temp_adiabat[key][1], rh[key][1])
        temp_lcl[key][1] = np.clip(lcl_temp_bolton(temp_surf[key][1], rh[key][1]*100), 0, temp_surf[key][1])
        for i in range(n_climates):
            temp_profile[key][i] = convection_neutral_profile(temp_surf[key][i], pressure_surface, temp_lcl[key][i], pressure_schematic)
            pressure_lcl[key][i] = dry_profile_pressure(temp_surf[key][i], pressure_surface, temp_lcl[key][i])
    
    # Plot profiles
    colors_day = {'cold': 'b', 'hot': 'r'}
    linestyles_climate = ['-', '--']
    if include_diff_plot:
        fig, ax = plt.subplots(1,2, figsize=(2*width['one_col'], width['one_col']/ar), sharey=True)
    else:
        fig, ax = plt.subplots(1,1, figsize=(width['one_col'], width['one_col']/ar), sharey=True)
        ax = [ax]
    for i in range(n_climates):
        for key in temp_surf:
            ax[0].plot(temp_profile[key][i], pressure_schematic/100, color=colors_day[key], linestyle=linestyles_climate[i])
            for j in range(len(ax)):
                ax[j].axhline(pressure_lcl[key][i]/100, color=colors_day[key], linestyle=linestyles_climate[i], lw=ax_linewidth)
        if include_diff_plot:
            ax[1].plot(temp_profile['hot'][i]-temp_profile['cold'][i], pressure_schematic/100, linestyle=linestyles_climate[i], color='k')
    for j in range(len(ax)):
        ax[j].axhline(pressure_ft/100, color='k', lw=ax_linewidth)
    ax[0].invert_yaxis()
    ax[0].set_ylim(np.round(pressure_levels[-1]/100, -2), pressure_levels[0]/100)
    ax[0].set_xlabel('Temperature [K]')
    ax[0].set_ylabel('Pressure [hPa]')
    if include_diff_plot:
        ax[1].set_xlabel('Temperature Anomaly, $\Delta T$ [K]')
    return fig, ax

### Hot Get Hotter
Between 900hPa and 750hPa, the two days follow a different lapse rate - this is the crucial effect, not the changing of the LCL.

In [39]:
# Hot get hotter - red line has higher temperature in ref climate
save_hot_hotter_fig = False
show_cold_hotter = False
temp_a_change_schematic = 30
fontsize_annotate = 9
# fig, ax = schematic_plot(temp_cold=280, sphum_cold=q_schematic, temp_adiabat_change_cold=temp_a_change_schematic, 
#                          temp_hot=290, include_diff_plot=False, rh_cold=None)

if show_cold_hotter:
# Cold get hotter
    fig, ax = schematic_plot(temp_cold=270, rh_cold=0.4, temp_adiabat_change_cold=temp_a_change_schematic, 
                             temp_hot=280, include_diff_plot=False)
    label_subplots(fig, ax, ['a)'], box_alpha=1)
    ax[0].annotate("$\Delta T_A$", xy=(227, (pressure_ft-1000)/100), ha='center', va='bottom', bbox=dict(facecolor='1', edgecolor='none', pad=0.0, alpha=1),
                   fontsize=fontsize_annotate)
    ax[0].annotate("", xy=(222, pressure_ft/100), 
                xytext=(234, pressure_ft/100),
                arrowprops=dict(arrowstyle="<->", ec='k', lw=0.5))
    
    ax[0].annotate("$\Delta T_s$", xy=(275, (pressure_surface-1000)/100), ha='center', va='bottom', bbox=dict(facecolor='1', edgecolor='none', pad=0.0, alpha=1),
                   fontsize=fontsize_annotate)
    ax[0].annotate("", xy=(269, pressure_surface/100), 
                xytext=(281, pressure_surface/100),
                arrowprops=dict(arrowstyle="<->", ec='k', lw=0.5))
    
    ax[0].annotate("$p_{FT}$", xy=(304, (pressure_ft-1000)/100), ha='center', va='bottom', bbox=dict(facecolor='1', edgecolor='none', pad=0.0, alpha=1),
                   fontsize=fontsize_annotate)
    ax[0].annotate("$p_{LCL}$", xy=(304, 790), ha='center', va='bottom', bbox=dict(facecolor='1', edgecolor='none', pad=0.0, alpha=1),
                   fontsize=fontsize_annotate)
else:
    # Hot get hotter
    fig, ax = schematic_plot(temp_cold=300, rh_cold=0.4, temp_adiabat_change_cold=temp_a_change_schematic, 
                             temp_hot=310, include_diff_plot=False)

if save_hot_hotter_fig or save_fig:
    fig.savefig(f"/Users/joshduffield/Desktop/schematic_{'cold' if show_cold_hotter else 'hot'}_hotter.pdf",
                dpi=dpi['combination'] if publish_fig else low_dpi, bbox_inches='tight', pad_inches=save_pad_inches)

In [6]:
fig, ax = schematic_plot(temp_cold=280, rh_cold=1, temp_adiabat_change_cold=temp_a_change_schematic, 
                         temp_hot=290, include_diff_plot=True)

### Drier Get Hotter
Below we see the amplified warming of dry days.

In the theory, there are two contributions: one positive and one negative. In the schematic we can see these two effects:
* Amplified warming of dry days: If drier, then LCL is at higher altitude so have region where dry days follow dry adiabat but moist days follow moist adiabat. It is this lapse rate differential than is crucial.
* Suppressed warming of dry days: If drier, temperature in free troposphere is lower. So moist adiabatic lapse rate is closer to dry adiabatic lapse rate than it is for moist days. This means the lapse rate differential is smaller on dry days than it is on moist days.

In [31]:
save_dry_hotter_fig = False
# fig, ax = schematic_plot(temp_cold=280, sphum_cold=q_schematic * 0.5, temp_adiabat_change_cold=temp_a_change_schematic, 
#                          sphum_hot=q_schematic*1.1, include_diff_plot=False)        # dry get hotter
# label_subplots(fig, ax, ['b)'], box_alpha=1)
# fig, ax = schematic_plot(temp_cold=320, sphum_cold=q_schematic * 0.5, temp_adiabat_change_cold=temp_a_change_schematic, 
#                          sphum_hot=q_schematic*1.3, include_diff_plot=False)      # moist get hotter

# fig, ax = schematic_plot(temp_cold=320, rh_cold=None, sphum_cold=sphum_sat(320, pressure_surface) * 0.8,
#                          temp_adiabat_change_cold=temp_a_change_schematic, 
#                          sphum_hot=sphum_sat(320, pressure_surface) * 0.9, include_diff_plot=True)      # moist get hotter

fig, ax = schematic_plot(temp_cold=280, rh_cold=0.2, rh_hot = 0.8,
                         temp_adiabat_change_cold=temp_a_change_schematic, include_diff_plot=False)      # dry get hotter
label_subplots(fig, ax, ['b)'], box_alpha=1)

 # moist get hotter not seen - I imagine because non-linear and squared terms are important in this case.
# fig, ax = schematic_plot(temp_cold=320, rh_cold=0.9, rh_hot = 0.95,
#                          temp_adiabat_change_cold=temp_a_change_schematic, include_diff_plot=True)     

if save_dry_hotter_fig or save_fig:
    fig.savefig(f"/Users/joshduffield/Desktop/schematic_dry_hotter.pdf",
                dpi=dpi['combination'] if publish_fig else low_dpi, bbox_inches='tight', pad_inches=save_pad_inches)

In [8]:
fig, ax = schematic_plot(temp_cold=290, rh_cold=0.95, rh_hot = 1,
                         temp_adiabat_change_cold=temp_a_change_schematic, include_diff_plot=True)      # dry get hotter

### Changing RH anomaly
Here, the changing of the LCL is the crucial effect, not the change of lapse rate with temperature.

In [34]:
save_rh_change_fig = False
fig, ax = schematic_plot(temp_cold=280, rh_cold=0.2, temp_adiabat_change_cold=temp_a_change_schematic, 
                         r_change_cold=0.4, include_diff_plot=False)
label_subplots(fig, ax, ['c)'], box_alpha=1)

if save_rh_change_fig or save_fig:
    fig.savefig(f"/Users/joshduffield/Desktop/schematic_rh_change.pdf",
                dpi=dpi['combination'] if publish_fig else low_dpi, bbox_inches='tight', pad_inches=save_pad_inches)

### Changing adiabatic temperature anomaly
Here, it is obvious that if you warm one day more in the free troposphere than another, it will lead to more warming at the surface if nothing else is changed.

The surface warming is suppressed compared to free troposphere though, this is because the moist adiabatic lapse rate is smaller for the day that is warmed more.

In [10]:
# Temp adiabat change - red line has larger change to adiabatic temperature
save_temp_a_change_fig = False
fig, ax = schematic_plot(temp_cold=280, rh_cold=0.4, temp_adiabat_change_cold=temp_a_change_schematic, 
                         temp_adiabat_change_hot=temp_a_change_schematic+10, include_diff_plot=False)
label_subplots(fig, ax, ['d)'], box_alpha=1)

if save_temp_a_change_fig or save_fig:
    fig.savefig(f"/Users/joshduffield/Desktop/schematic_temp_a_change.pdf",
                dpi=dpi['combination'] if publish_fig else low_dpi, bbox_inches='tight', pad_inches=save_pad_inches)

### Lapse Rate Comparison
The key to the schematics above is that there is a pressure interval where one day follows moist adiabat and one follows dry adiabat. Below I show that the gradient of the dry adiabat increases slightly with warming (becomes less steep in above figures) while the moist adiabatic lapse rate decreases (becomes more steep). It is this different lapse rate response to warming that is critical.

In [37]:
save_lapse_fig = False
lapse_plot_pressure = 600 * 100
lapse_plot_temp = np.linspace(220, 320, 300)
lapse_plot_z_coords = False

fig, ax = plt.subplots(1,1, figsize=(width['one_col'], width['one_col']/ar), sharex=True, sharey=True)
if lapse_plot_z_coords:
    ax.plot(lapse_plot_temp, lapse_moist(lapse_plot_temp, lapse_plot_pressure) * 1000, color='k', label='Moist')
    ax.axhline(lapse_dry * 1000, color='k', label='Dry', linestyle='--')
    ax.set_ylabel('Lapse rate, $-dT/dz [K/km]$')
else:
    ax.plot(lapse_plot_temp, lapse_moist(lapse_plot_temp, lapse_plot_pressure, True) * 100, color='k', label='Moist')
    ax.plot(lapse_plot_temp, lapse_dry * R * lapse_plot_temp / (lapse_plot_pressure * g) * 100, color='k', label='Dry', linestyle='--')
    ax.set_ylabel('Lapse rate, $dT/dp$ [K/hPa]')
ax.legend()
ax.set_xlim(lapse_plot_temp[0], lapse_plot_temp[-1])
ax.set_xlabel('Temperature [K]')

# Cold get hotter - lapse rate decreases more for hot day than cold day
temp_change_use = 15
temp_day_diff_use = 10
marker_use = ['.', '+']
marker_size = 30
for i, temp_cold_use in enumerate([237, 270]):
    ax.scatter(temp_cold_use, lapse_moist(temp_cold_use, lapse_plot_pressure, True)*100, color='b', marker=marker_use[i],
               s=marker_size, zorder=3)   # zorder brings markers to front
    ax.scatter(temp_cold_use+temp_change_use, lapse_moist(temp_cold_use+temp_change_use, lapse_plot_pressure, True)*100, 
               color='b', marker=marker_use[i], s=marker_size, zorder=3)
    ax.scatter(temp_cold_use+temp_day_diff_use, lapse_moist(temp_cold_use+temp_day_diff_use, lapse_plot_pressure, True)*100, color='r',
               marker=marker_use[i], s=marker_size, zorder=3)
    ax.scatter(temp_cold_use+temp_day_diff_use+temp_change_use, 
               lapse_moist(temp_cold_use+temp_day_diff_use+temp_change_use, lapse_plot_pressure, True)*100, 
               color='r', marker=marker_use[i], s=marker_size, zorder=3)

if save_fig or save_lapse_fig:
    fig.savefig(f"/Users/joshduffield/Desktop/lapse_schematic.pdf",
                dpi=dpi['combination'] if publish_fig else low_dpi, bbox_inches='tight', pad_inches=save_pad_inches)

## Non-linear nature of $q^*$ Schematic
Solid line shows $q^*(T)$, dotted line shows $q^*(T+\epsilon)$ with $\epsilon=15K$.

In [12]:
schematic_q_temp_values = np.linspace(250, 330, 50)
schematic_q_values = sphum_sat(schematic_q_temp_values, pressure_ft)             # use free troposphere pressure as illustrating temp adiabat
schematic_q_epsilon = 15
schematic_q_values_shift = sphum_sat(schematic_q_temp_values+schematic_q_epsilon, pressure_ft)

save_schematic_q_sat_fig = False
fig, ax = plt.subplots(1, 1, figsize=(width['one_col'], width['one_col'] / ar))
ax.plot(schematic_q_temp_values, schematic_q_values, color='k', lw=2)
ax.plot(schematic_q_temp_values, schematic_q_values_shift, color='k', linestyle=':', lw=2)
ax.set_xlim(schematic_q_temp_values[0], schematic_q_temp_values[-1])
ax.set_ylabel('$q^*$ [kg/kg]')
ax.set_xlabel('$T$ [K]')
ax.set_yticks([0, 0.05, 0.1, 0.15, 0.2])
ax.set_xticks([250, 270, 290, 310, 330])
ax.set_ylim(0, 0.2)
schematic_arrow_info = {'cold': [280, 'blue'], 'mean': [290, 'green'], 'hot': [300, 'red']}
for key in schematic_arrow_info:
    schematic_day_ind = np.argmin(np.abs(schematic_q_temp_values-schematic_arrow_info[key][0]))
    ax.annotate("", xy=(schematic_q_temp_values[schematic_day_ind], schematic_q_values[schematic_day_ind]), 
                xytext=(schematic_q_temp_values[schematic_day_ind], schematic_q_values_shift[schematic_day_ind]),
                arrowprops=dict(arrowstyle="<-", ec=schematic_arrow_info[key][1], lw=1))

if save_schematic_q_sat_fig or save_fig:
    fig.savefig(f"/Users/joshduffield/Desktop/schematic_q_sat.pdf",
                dpi=dpi['combination'] if publish_fig else low_dpi, bbox_inches='tight', pad_inches=save_pad_inches)

## Prefactor for each term
Here I plot the prefactor for each term as a function of near-surface conditions. This indicates the sensitivity to that particular mechanism for different climates. 

In the plots below, red means a strong warming effect and blue means a strong cooling effect for a positive $\delta$ or $\Delta$ term.

In [13]:
n_prefactor_temp = 81
n_prefactor_rh = 101
temp_plot_values = np.linspace(235, 315, n_prefactor_temp)
rh_plot_values = np.linspace(0, 1, n_prefactor_rh)
temp_plot_values_2d = np.repeat(temp_plot_values[:, np.newaxis], axis=1, repeats=n_prefactor_rh)
# temp_plot_values = np.linspace(297.73, 297.732, n_prefactor_temp)
# rh_plot_values = np.linspace(0.55, 0.67, n_prefactor_rh)
# prefactor_plot_temp_adiabat = np.zeros((n_prefactor_temp, n_prefactor_rh))
sphum_plot_values = np.zeros((n_prefactor_temp, n_prefactor_rh))
gamma = {key: np.zeros((n_prefactor_temp, n_prefactor_rh)) for key in ['temp_s_r', 'temp_s_q', 'humidity', 'rh_change', 'temp_a_change']}
beta_ratio = np.zeros((n_prefactor_temp, n_prefactor_rh))

for i in range(n_prefactor_temp):
    for j in range(n_prefactor_rh):
        sphum_use = rh_plot_values[j] * sphum_sat(temp_plot_values[i], pressure_surface)
        # prefactor_plot_temp_adiabat[i, j] = get_temp_adiabat(temp_plot_values[i], 
        #                                                      sphum_plot_values[i, j],
        #                                                      pressure_surface, pressure_ft)
        gamma['temp_s_r'][i, j], gamma['humidity'][i, j], gamma['rh_change'][i, j], gamma['temp_a_change'][i, j] = \
            get_gamma_factors(temp_plot_values[i], sphum_use, pressure_surface, pressure_ft)
        gamma['temp_s_q'][i, j] = get_gamma_factors(temp_plot_values[i], sphum_use, pressure_surface, pressure_ft, True)[0]
        # get ratio of beta_s1/(beta_s1+beta_a1) as gamma factors multiplied by this when used in extratropical z form of theory.
        beta_s1_mean = get_theory_prefactor_terms(temp_plot_values[i], pressure_surface, pressure_ft, sphum_use)[3]
        temp_adiabat = get_temp_adiabat(temp_plot_values[i], sphum_use, pressure_surface, pressure_ft)
        beta_a1 = get_theory_prefactor_terms(temp_plot_values[i], pressure_surface, pressure_ft)[3]
        beta_ratio[i, j] = beta_s1_mean/(beta_s1_mean+beta_a1)

In [ ]:
def gamma_plot(gamma_factor, temp_values=temp_plot_values, rh_values=rh_plot_values, ref_temp_value=275, ref_rh_value=0.5, 
               cbar_label='$\gamma/|\gamma_{ref}|$', norm=True):
    # Plot a colormap indicating how sensitivty parameter varies with mean temperature and relative humidity of current climate.
    ref_ind_temp = np.argmin(np.abs(temp_values-ref_temp_value))
    ref_ind_rh = np.argmin(np.abs(rh_values-ref_rh_value))
    
    # Normalise prefactor by a reference value
    if norm:
        gamma_norm = gamma_factor / np.abs(gamma_factor[ref_ind_temp, ref_ind_rh])
    else:
        gamma_norm = gamma_factor
    
    fig, ax = plt.subplots(1, 1, figsize=(width['one_col'], width['one_col'] / ar))
    im = ax.pcolormesh(rh_values*100, temp_values, gamma_norm, cmap='turbo', shading='gouraud')
    cbar = fig.colorbar(im, ax=ax, label=cbar_label)
    ax.set_xlim(rh_values[0]*100, rh_values[-1]*100)
    ax.set_ylim(temp_values[0], temp_values[-1])
    ax.set_xlabel('Relative humidity, $\overline{r_s}$ [%]')
    ax.set_ylabel('Temperature, $\overline{T_s}$ [K]')
    if gamma_norm.max() * gamma_norm.min() < 0:
        # If sign of prefactor changes, then hatch the positive values
        plt.rcParams['hatch.linewidth'] = 0.1
        # ax.scatter(rh_values[np.where(prefactor_norm>0)[1]]*100, temp_values[np.where(prefactor_norm>0)[0]], marker='+', color='k', s=0.6)
        # ax.fill_between(rh_plot_values*100, temp_values[np.argmin(np.abs(prefactor), axis=0)], 
        #                 np.full_like(rh_values, temp_values[-1]), color="none",hatch="xx",edgecolor="k", lw=0.1)
        # ax.scatter()
        ax.contour(rh_values*100, temp_values, gamma_norm, [0], colors='k', linewidths=0.5)
        # cbar.ax.fill_between([0, 1], [0, 0], [prefactor_norm.max(), prefactor_norm.max()], color="none",hatch="xx",edgecolor="k", lw=0.1)
        # cbar.ax.contour(rh_values*100, temp_values, prefactor_norm, [0], colors='k', linewidths=0.5)
        cbar.ax.axhline(0, color='k', lw=0.5)
    return fig, ax

In [ ]:
# Plot simulation values for each kappa
temp_sim_trop_summer_mean_values = np.array([291.33965875, 294.55063238, 297.00180021, 301.42089629, 
                                             304.45573939, 306.79560786, 308.65814601, 310.30251747])
rh_sim_trop_summer_mean_values = np.array([0.61858373, 0.62144154, 0.62784442, 0.64272708, 0.65531253, 0.66727618, 0.67744756, 0.68635731]) * 100

temp_sim_trop_winter_mean_values = np.array([285.94924521, 289.59407875, 292.36411124, 297.26257061,
                                             300.60884691, 303.21498673, 305.27064183, 307.14459597])
rh_sim_trop_winter_mean_values = np.array([0.61593213, 0.62303681, 0.63496396, 0.66429798, 0.68528345,
                                           0.70187875, 0.71370379, 0.72466047]) * 100    # done

temp_sim_extrop_summer_mean_values = np.array([292.51546328, 295.99569731, 298.67885426, 303.1859222 , 
                                               306.17776398, 308.24069069, 309.97902923, 311.34886395])
rh_sim_extrop_summer_mean_values = np.array([0.6856265 , 0.66904473, 0.65862026, 0.65246149, 0.65492535, 
                                             0.6626966 , 0.6674643 , 0.67197353]) * 100   # done

temp_sim_extrop_winter_mean_values = np.array([237.31051576, 244.61846976, 250.35618013, 260.02567979,
                                               267.17754475, 272.25164051, 276.42424502, 279.72619096])
rh_sim_extrop_winter_mean_values = np.array([0.85721708, 0.80275693, 0.76997156, 0.73091624, 0.71309605, 
                                             0.7104835 , 0.71529052, 0.72063939]) * 100   # done

In [ ]:
humidity_use = 'r'

save_gamma_temp_s_fig = False
fig, ax = gamma_plot(gamma['temp_s_'+humidity_use])
ax.scatter(rh_sim_trop_summer_mean_values, temp_sim_trop_summer_mean_values, color='w', marker='+', s=20)
ax.scatter(rh_sim_trop_winter_mean_values, temp_sim_trop_winter_mean_values, color='w', marker='.', s=10)
ax.scatter(rh_sim_extrop_summer_mean_values, temp_sim_extrop_summer_mean_values, color='grey', marker='+', s=20)
ax.scatter(rh_sim_extrop_winter_mean_values, temp_sim_extrop_winter_mean_values, color='grey', marker='.', s=10)
label_subplots(fig, [ax], ['a)'], box_alpha=0, fontcolor='k')

if save_gamma_temp_s_fig or save_fig:
    fig.savefig(f"/Users/joshduffield/Desktop/gamma_temp_s_{humidity_use}.pdf",
                dpi=dpi['combination'] if publish_fig else low_dpi, bbox_inches='tight', pad_inches=save_pad_inches)

In [ ]:
save_gamma_humidity_fig = False
fig, ax = gamma_plot(gamma['humidity'])
ax.scatter(rh_sim_trop_summer_mean_values, temp_sim_trop_summer_mean_values, color='w', marker='+', s=20)
ax.scatter(rh_sim_trop_summer_mean_values, temp_sim_trop_winter_mean_values, color='w', marker='.', s=10)
ax.scatter(rh_sim_extrop_summer_mean_values, temp_sim_extrop_summer_mean_values, color='grey', marker='+', s=20)
ax.scatter(rh_sim_extrop_winter_mean_values, temp_sim_extrop_winter_mean_values, color='grey', marker='.', s=10)
label_subplots(fig, [ax], ['b)'], box_alpha=0, fontcolor='k')

if save_gamma_humidity_fig or save_fig:
    fig.savefig(f"/Users/joshduffield/Desktop/gamma_humidity_{humidity_use}.pdf",
                dpi=dpi['combination'] if publish_fig else low_dpi, bbox_inches='tight', pad_inches=save_pad_inches)

In [ ]:
save_gamma_rh_change_fig = False
fig, ax = gamma_plot(gamma['rh_change'])
ax.scatter(rh_sim_trop_summer_mean_values, temp_sim_trop_summer_mean_values, color='w', marker='+', s=20)
ax.scatter(rh_sim_trop_summer_mean_values, temp_sim_trop_winter_mean_values, color='w', marker='.', s=10)
ax.scatter(rh_sim_extrop_summer_mean_values, temp_sim_extrop_summer_mean_values, color='grey', marker='+', s=20)
ax.scatter(rh_sim_extrop_winter_mean_values, temp_sim_extrop_winter_mean_values, color='grey', marker='.', s=10)
label_subplots(fig, [ax], ['c)'], box_alpha=0, fontcolor='k')

if save_gamma_rh_change_fig or save_fig:
    fig.savefig(f"/Users/joshduffield/Desktop/gamma_rh_change.pdf",
                dpi=dpi['combination'] if publish_fig else low_dpi, bbox_inches='tight', pad_inches=save_pad_inches)

In [ ]:
save_gamma_temp_a_change_fig = False
fig, ax = gamma_plot(gamma['temp_a_change'])
ax.scatter(rh_sim_trop_summer_mean_values, temp_sim_trop_summer_mean_values, color='w', marker='+', s=20)
ax.scatter(rh_sim_trop_summer_mean_values, temp_sim_trop_winter_mean_values, color='w', marker='.', s=10)
ax.scatter(rh_sim_extrop_summer_mean_values, temp_sim_extrop_summer_mean_values, color='grey', marker='+', s=20)
ax.scatter(rh_sim_extrop_winter_mean_values, temp_sim_extrop_winter_mean_values, color='grey', marker='.', s=10)
label_subplots(fig, [ax], ['d)'], box_alpha=0, fontcolor='k')

if save_gamma_temp_a_change_fig or save_fig:
    fig.savefig(f"/Users/joshduffield/Desktop/gamma_temp_a_change.pdf",
                dpi=dpi['combination'] if publish_fig else low_dpi, bbox_inches='tight', pad_inches=save_pad_inches)

In [ ]:
# Gamma factors are multiplied by this when using z form of theory
save_beta_ratio_fig = False
fig, ax = gamma_plot(beta_ratio, norm=False, cbar_label='$\overline{\\beta_{s1}}/(\overline{\\beta_{s1}} + \\beta_{A1})$')
ax.scatter(rh_sim_trop_summer_mean_values, temp_sim_trop_summer_mean_values, color='w', marker='+', s=20)
ax.scatter(rh_sim_trop_summer_mean_values, temp_sim_trop_winter_mean_values, color='w', marker='.', s=10)
ax.scatter(rh_sim_extrop_summer_mean_values, temp_sim_extrop_summer_mean_values, color='grey', marker='+', s=20)
ax.scatter(rh_sim_extrop_winter_mean_values, temp_sim_extrop_winter_mean_values, color='grey', marker='.', s=10)

if save_beta_ratio_fig or save_fig:
    fig.savefig(f"/Users/joshduffield/Desktop/beta_ratio.pdf",
                dpi=dpi['combination'] if publish_fig else low_dpi, bbox_inches='tight', pad_inches=save_pad_inches)

## Contribution to scale factor - variation with current climate
For a fairly typical case with $\kappa=1$ to $\kappa=1.5$ for $x=99$ in tropical summer, how does the scale factor $\delta T_s(x)/\delta \overline{T_s}$ change with the mean conditions of the current climate i.e. $\overline{T_s}$ and $\overline{r_s}$?

Secondly how does the fractional contribution of each mechanism vary?

In [ ]:
# kappa=1 to 1.5 values for tropical summer, x=99
temp_s_anom_use = 3.032476607127886
temp_s_mean_use = 297.0018002088079
rh_anom_use = -0.02989558253938038
rh_mean_use = 0.6278444162988804
rh_anom_change_use = -0.008537580800209237
temp_a_anom_change_use = -0.6912736802929262
temp_mean_change_use = 4.419096084885098

sf_cont_temp_s = gamma['temp_s_r'] * temp_s_anom_use/temp_s_mean_use
sf_cont_humidity = gamma['humidity'] * rh_anom_use/rh_mean_use
sf_cont_rh_change = gamma['rh_change'] * rh_anom_change_use/rh_mean_use/temp_mean_change_use
sf_cont_temp_a_change = gamma['temp_a_change'] * temp_a_anom_change_use/temp_mean_change_use

In [ ]:
save_scale_factor_fig = False
fig, ax = plt.subplots(1, 1, figsize=(width['one_col'], width['one_col'] / ar))
im = ax.pcolormesh(rh_plot_values*100, temp_plot_values, 1+sf_cont_temp_s + sf_cont_humidity + sf_cont_rh_change + sf_cont_temp_a_change, 
                    cmap='turbo', vmax=1, vmin=0.8, shading='gouraud')
cbar = plt.colorbar(im, label='Scaling factor, $\delta T_s(x)/\delta \overline{T_s}$')
cbar.ax.set_yticks([0.8, 0.85, 0.9, 0.95, 1])
ax.scatter(rh_mean_use*100, temp_s_mean_use, color='w', marker='+', s=20)
ax.set_xlabel('Relative humidity, $\overline{r_s}$ [%]')
ax.set_ylabel('Temperature, $\overline{T_s}$ [K]')
if save_scale_factor_fig:
    fig.savefig(f"/Users/joshduffield/Desktop/scale_factor_mean_climate.pdf",
                dpi=dpi['combination'] if publish_fig else low_dpi, bbox_inches='tight', pad_inches=save_pad_inches)

In [ ]:
var = [sf_cont_temp_s, sf_cont_humidity, sf_cont_rh_change, sf_cont_temp_a_change]
abs_sum = np.sum(np.asarray([np.abs(var_i) for var_i in var]), axis=0)
var_i = 0

save_scale_factor_cont_fig = False
fig, ax = plt.subplots(1, 1, figsize=(width['one_col'], width['one_col'] / ar))
im = ax.pcolormesh(rh_plot_values*100, temp_plot_values, 100*np.abs(var[var_i])/abs_sum, 
                    cmap='turbo', vmin=0, vmax=100, shading='gouraud')
if var_i <= 1:
    ax.contour(rh_plot_values*100, temp_plot_values, var[var_i], [0], colors='w', linewidths=0.5)
label_subplots(fig, [ax], [chr(97+var_i)+')'], box_alpha=0, fontcolor='w')
cbar = plt.colorbar(im, label='Scaling factor contribution [%]')
# cbar.ax.set_yticks([0.8, 0.85, 0.9, 0.95, 1])
ax.scatter(rh_mean_use*100, temp_s_mean_use, color='w', marker='+', s=20)
ax.set_xlabel('Relative humidity, $\overline{r_s}$ [%]')
ax.set_ylabel('Temperature, $\overline{T_s}$ [K]')
if save_scale_factor_cont_fig:
    fig.savefig(f"/Users/joshduffield/Desktop/scale_factor_mean_climate_cont{var_i}.pdf",
                dpi=dpi['combination'] if publish_fig else low_dpi, bbox_inches='tight', pad_inches=save_pad_inches)